### Utilizando subquery no SQL

In [1]:
# Importando o pandas
import pandas as pd

In [2]:
# Criando a conexão
import sqlite3
con = sqlite3.connect('BaseDados.db')

In [3]:
# Criando um cursor
cur = con.cursor()

In [4]:
# Criando uma função para consultar os dados
def executa_sql(comando):
    cur.execute(comando)
    resultado = cur.fetchall()
    resultado = pd.DataFrame(resultado)
    if resultado.shape[1] > 0:
        resultado.columns = [i[0] for i in cur.description]
    print(resultado.shape)
    display(resultado.head())
    return resultado

In [5]:
# Utilizando parte do CASE que vimos na última aula
resultado_sql = executa_sql('SELECT nome_aluno, cod_matricula, \
                            AVG(nota_prova) as media_prova, \
                            count(nota_prova) as qtd_provas, \
                            (CASE \
                                WHEN AVG(nota_prova) > 7 THEN "Aprovado" \
                                WHEN AVG(nota_prova) > 5 AND count(nota_prova) = 1 THEN "Fazer prova 2" \
                            END) as situacao_aluno \
                            FROM dados \
                            WHERE nota_prova IS NOT NULL \
                            GROUP BY nome_aluno,cod_matricula \
                            ORDER BY AVG(nota_prova) desc')

(15, 5)


,nome_aluno,cod_matricula,media_prova,qtd_provas,situacao_aluno
0,Bárbara da Cunha,63546,10.0,1,Aprovado
1,Bárbara Freitas,19442,8.0,2,Aprovado
2,Gabriela Costela,21262,8.0,1,Aprovado
3,Júlia Pinto,47086,8.0,2,Aprovado
4,Lívia Jesus,22284,8.0,1,Aprovado


### SUBQUERY
- No SQL conseguimos fazer uma "query dentro da query", o que chamamos de subquery
- Isso será muito útil quando falarmos de junção entre várias tabelas e quando utilizamos cálculos, como no caso do CASE da última aula

In [6]:
# Transformando a query sem o CASE em uma subquery
resultado_sql = executa_sql('SELECT nome_aluno, cod_matricula, media_prova, qtd_provas, \
                            (CASE \
                                WHEN media_prova > 7 THEN "Aprovado" \
                                WHEN media_prova > 5 AND qtd_provas = 1 THEN "Fazer prova 2" \
                             END) as situacao_aluno \
                            FROM (SELECT nome_aluno, cod_matricula, \
                            AVG(nota_prova) as media_prova, \
                            count(nota_prova) as qtd_provas \
                            FROM dados \
                            WHERE nota_prova IS NOT NULL \
                            GROUP BY nome_aluno,cod_matricula \
                            ORDER BY AVG(nota_prova) desc)')

(15, 5)


,nome_aluno,cod_matricula,media_prova,qtd_provas,situacao_aluno
0,Bárbara da Cunha,63546,10.0,1,Aprovado
1,Bárbara Freitas,19442,8.0,2,Aprovado
2,Gabriela Costela,21262,8.0,1,Aprovado
3,Júlia Pinto,47086,8.0,2,Aprovado
4,Lívia Jesus,22284,8.0,1,Aprovado


In [12]:
# Utilizando a subquery para fazer o CASE ENCAPSULAMENTO
# exemplo de subquery -> resultado_sql = executa_sql('SELECT * FROM ( SELECT * FROM dados))

resultado_sql = executa_sql('SELECT nome_aluno, cod_matricula, media_prova, qtd_provas, \
                            (CASE \
                                WHEN media_prova > 7 THEN "APROVADO" \
                                WHEN media_prova > 5 AND qtd_provas = 1 THEN "FAZER PROVA 2" \
                                WHEN media_prova > 5 AND qtd_provas = 2 THEN "FAZER PROVA FINAL" \
                                WHEN media_prova > 5 AND qtd_provas > 2 THEN "REVISAR MATÉRIA" \
                                WHEN media_prova < 5 AND qtd_provas = 1  THEN "REVISAR MATÉRIA" \
                                WHEN media_prova < 5 AND qtd_provas > 1 THEN "LISTA DE EXERCÍCIOS" \
                                ELSE "VERIFICAR CÓDIGO" \
                             END) as situacao_aluno \
                            FROM (SELECT nome_aluno, cod_matricula, \
                            (AVG(nota_prova)+0.5) as media_prova, \
                            count(nota_prova) as qtd_provas \
                            FROM dados \
                            WHERE nota_prova IS NOT NULL \
                            GROUP BY nome_aluno,cod_matricula \
                            ORDER BY AVG(nota_prova) desc)')

(15, 5)


,nome_aluno,cod_matricula,media_prova,qtd_provas,situacao_aluno
0,Bárbara da Cunha,63546,10.5,1,APROVADO
1,Bárbara Freitas,19442,8.5,2,APROVADO
2,Gabriela Costela,21262,8.5,1,APROVADO
3,Júlia Pinto,47086,8.5,2,APROVADO
4,Lívia Jesus,22284,8.5,1,APROVADO


In [13]:
display(resultado_sql)

,nome_aluno,cod_matricula,media_prova,qtd_provas,situacao_aluno
0,Bárbara da Cunha,63546,10.500000,1,APROVADO
1,Bárbara Freitas,19442,8.500000,2,APROVADO
2,Gabriela Costela,21262,8.500000,1,APROVADO
3,Júlia Pinto,47086,8.500000,2,APROVADO
4,Lívia Jesus,22284,8.500000,1,APROVADO
5,Melissa Ribeiro,38438,8.500000,2,APROVADO
6,Maria Eduarda da Rocha,38273,8.166667,3,APROVADO
7,Eloah Aragão,65749,8.000000,4,APROVADO
8,Isabelly Souza,31859,7.500000,1,APROVADO
9,Kevin Melo,80515,7.500000,1,APROVADO
